In [ ]:
from transformers.models import gpt_neo
from transformers import GPT2Tokenizer
import torch
import torch.nn.functional as F
import deepspeed
import os

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9868' # modify if RuntimeError: Address already in use
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

In [ ]:
torch.distributed.init_process_group(backend="gloo")

In [ ]:
deepspeed.init_distributed("gloo")
device = torch.device("cuda", 0)

In [ ]:
model = gpt_neo.GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

In [ ]:
model.requires_grad_(False)
os.system("cls")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
config = {
  "train_batch_size": 1,
  "zero_optimization": {
   "stage": 0,
    "offload_optimizer": {
      "device": "cpu"
    },
    "offload_param": {
      "device": "cpu"
    }
  },
  "fp16": {
    "enabled": True
  }
}

model_engine, optimizer, _, _ = deepspeed.initialize(model=model,
                                                     config_params=config)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def top_k_logits(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ This function has been mostly taken from huggingface conversational
     ai code at
         https://medium.com/huggingface/how-to-build-a-state-of-the-art-
              conversational-ai-with-transfer-learning-2d818ac26313 """

    if top_k > 0:
        # Remove all tokens with a probability less than the
        # last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Cconvert to 1D
        sorted_logits, sorted_indices = torch.sort(
            logits, descending=True, dim=-1)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1),
                                        dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token
        # above the threshold
        sorted_indices_to_remove[..., 1:] \
            = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        for i in range(sorted_indices.size(0)):
            indices_to_remove = sorted_indices[i][sorted_indices_to_remove[i]]
            logits[i][indices_to_remove] = filter_value

    return logits

In [ ]:
# Deepspeed doesn't have a generate method, so i created mine.
# Well... i just modified https://github.com/microsoft/DeepSpeedExamples/blob/127372571189ac905c8c92f4fe55a3d85c80324e/Megatron-LM-v1.1.5-3D_parallelism/megatron/text_generation_utils.py#L32
def generate(model,
             tokenizer,
             device,
             tokens,
             out_seq_length=50,
             temperature=1,
             top_k=0,
             top_p=0):

    tokens = tokens.input_ids.to(device)
    context_length = len(tokens[0])
    eos_id = tokenizer.eos_token_id
    counter = 0

    tokens = F.pad(tokens, (0,out_seq_length), value=2)

    for i in range(out_seq_length):
        logits = model(tokens).logits
        logits = logits[:, context_length - 1, :] / temperature
        logits = top_k_logits(logits, top_k=top_k, top_p=top_p)
        log_probs = torch.nn.functional.softmax(logits, dim=-1)
        prev = torch.multinomial(log_probs, num_samples=1)
        tokens[0][context_length] = prev[0]
        context_length += 1

        if eos_id in tokens[0]:
            break

    return tokens

In [ ]:
prompt = "You are Alex, a knight living in the kingdom of Larion. You have a steel longsword and a wooden shield. You are on a quest to defeat the evil dragon of Larion. You've heard he lives up at the north of the kingdom. You set on the path to defeat him and walk into a dark forest. As you enter the forest you see"
in_tokens = tokenizer(prompt, return_tensors="pt")
out_tokens = generate(model_engine, tokenizer, device, in_tokens, out_seq_length=50, top_k=60, top_p=0.9)[0]
decoded = tokenizer.decode(out_tokens)
print(decoded)